In [1]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join
from cobra.io import write_sbml_model

In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
data_dir = "/Users/lizrad/Dev/iVnat"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model=cobra.io.read_sbml_model(join(data_dir, "iVnat.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
model

files found: 
iVnat.xml, iVnat_mapped.xml


Name,GCF_001456255_1_rast_metabolic_model
Memory address,0x010e771d30
Number of metabolites,1842
Number of reactions,1906
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol_0, Extracellular_0"


### Read chem_xref

In [4]:
chem_xref = read_csv("/Users/lizrad/Dev/iVnat/chem_xref.tsv" , sep='\t' , low_memory=False)
chem_xref

,XREF,XREF_ID,MNX_ID,Evidence,Description
0,BIOMASS,BIOMASS,inferred,biomass,NaN
1,Biomass,BIOMASS,inferred,biomass,NaN
2,metanetx,MNXM01,MNXM01,identity,H(+)
3,bigg,10fthf,MNXM237,inferred,10-Formyltetrahydrofolate
4,metanetx,MNXM237,MNXM237,identity,NaN
5,bigg,10fthf5glu,MNXM3428,identity,10fthf5glu
6,metanetx,MNXM3428,MNXM3428,identity,NaN
7,bigg,10fthf6glu,MNXM3429,identity,10fthf6glu
8,metanetx,MNXM3429,MNXM3429,identity,NaN
9,bigg,10fthf7glu,MNXM5422,identity,10fthf7glu


### Bigg IDs to dict key

In [5]:
df_bigg = chem_xref.loc[2:10216,'XREF':'MNX_ID']
pivoted_bigg = df_bigg.pivot_table(index='XREF',columns='XREF_ID',values='MNX_ID',aggfunc = lambda x: x)
pivoted_bigg
#pivoted_bigg=pivoted_bigg.drop(['metanetx'])
chem_xref_MNX_bigg_dict = pivoted_bigg.to_dict('index')
chem_xref_MNX_bigg_dict2 = chem_xref_MNX_bigg_dict['bigg']
chem_xref_MNX_bigg_dict2

XREF_ID,10fthf,10fthf5glu,10fthf6glu,10fthf7glu,10fthfglu__L,11_cis_retfa,11docrtsl,11docrtstrn,12HPET,12RHPET,...,xylu__L,zcarote,zeax,zn2,zxan,zym_int1,zym_int2,zymst,zymstest_SC,zymstnl
XREF,,,,,,,,,,,,,,,,,,,,,
bigg,MNXM237,MNXM3428,MNXM3429,MNXM5422,MNXM164047,MNXM146564,MNXM163103,MNXM162887,MNXM13989,MNXM13994,...,MNXM597,MNXM1039,MNXM403,MNXM149,MNXM3903,MNXM91328,MNXM97119,MNXM574,MNXM147334,MNXM2494
metanetx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


{'10fthf': 'MNXM237',
 '10fthf5glu': 'MNXM3428',
 '10fthf6glu': 'MNXM3429',
 '10fthf7glu': 'MNXM5422',
 '10fthfglu__L': 'MNXM164047',
 '11_cis_retfa': 'MNXM146564',
 '11docrtsl': 'MNXM163103',
 '11docrtstrn': 'MNXM162887',
 '12HPET': 'MNXM13989',
 '12RHPET': 'MNXM13994',
 '12d3k5m': 'MNXM163631',
 '12dag3p_BS': 'MNXM163072',
 '12dgr120': 'MNXM4939',
 '12dgr140': 'MNXM146479',
 '12dgr141': 'MNXM4940',
 '12dgr160': 'MNXM3132',
 '12dgr16018111Z': 'MNXM145578',
 '12dgr1601819Z': 'MNXM145574',
 '12dgr161': 'MNXM3774',
 '12dgr180': 'MNXM4217',
 '12dgr1801819Z': 'MNXM145575',
 '12dgr181': 'MNXM3775',
 '12dgr18111Z160': 'MNXM147342',
 '12dgr18111Z18111Z': 'MNXM145577',
 '12dgr18111Z1819Z': 'MNXM145566',
 '12dgr1819Z160': 'MNXM146745',
 '12dgr1819Z1619Z': 'MNXM147343',
 '12dgr1819Z18111Z': 'MNXM145573',
 '12dgr1819Z1819Z': 'MNXM145565',
 '12dgr181_9': 'MNXM147348',
 '12dgr182_9_12': 'MNXM147347',
 '12dgr183_6_9_12': 'MNXM147345',
 '12dgr183_9_12_15': 'MNXM147346',
 '12dgr184_6_9_12_15': 'MNXM14

### Bigg IDs to annotation field

In [6]:
for met in model.metabolites:
    data=''
    if 'metanetx.chemical' in met.annotation:
        data=met.annotation['metanetx.chemical'] 
        
        for xref in chem_xref_MNX_bigg_dict2.keys():
            if chem_xref_MNX_bigg_dict2[xref]==data:
                met.annotation['bigg.metabolite']=xref
                print(met.annotation)

{'metanetx.chemical': 'MNXM421', 'seed.compound': 'cpd00443', 'bigg.metabolite': '4abz'}
{'metanetx.chemical': 'MNXM1160', 'seed.compound': 'cpd02920', 'bigg.metabolite': '2ahhmd'}
{'metanetx.chemical': 'MNXM1160', 'seed.compound': 'cpd02920', 'bigg.metabolite': '6hmhptpp'}
{'metanetx.chemical': 'MNXM11', 'seed.compound': 'cpd00012', 'bigg.metabolite': 'ppi'}
{'metanetx.chemical': 'MNXM1', 'seed.compound': 'cpd00067', 'bigg.metabolite': 'h'}
{'metanetx.chemical': 'MNXM965', 'seed.compound': 'cpd00683', 'bigg.metabolite': 'dhpt'}
{'metanetx.chemical': 'MNXM3', 'seed.compound': 'cpd00002', 'bigg.metabolite': 'atp'}
{'metanetx.chemical': 'MNXM29', 'seed.compound': 'cpd00033', 'bigg.metabolite': 'gly'}
{'metanetx.chemical': 'MNXM412', 'seed.compound': 'cpd00506', 'bigg.metabolite': 'glucys'}
{'metanetx.chemical': 'MNXM7', 'seed.compound': 'cpd00008', 'bigg.metabolite': 'adp'}
{'metanetx.chemical': 'MNXM9', 'seed.compound': 'cpd00009', 'bigg.metabolite': 'pi'}
{'metanetx.chemical': 'MNXM57'

{'metanetx.chemical': 'MNXM55269', 'seed.compound': 'cpd11589', 'bigg.metabolite': 'gly_asp__L'}
{'metanetx.chemical': 'MNXM51', 'seed.compound': 'cpd00038', 'bigg.metabolite': 'gtp'}
{'metanetx.chemical': 'MNXM58', 'seed.compound': 'cpd00048', 'bigg.metabolite': 'so4'}
{'metanetx.chemical': 'MNXM30', 'seed.compound': 'cpd00031', 'bigg.metabolite': 'gdp'}
{'metanetx.chemical': 'MNXM287', 'seed.compound': 'cpd00193', 'bigg.metabolite': 'aps'}
{'metanetx.chemical': 'MNXM266', 'seed.compound': 'cpd00206', 'bigg.metabolite': 'dcmp'}
{'metanetx.chemical': 'MNXM1777', 'seed.compound': 'cpd02227', 'bigg.metabolite': 'rhcys'}
{'metanetx.chemical': 'MNXM123', 'seed.compound': 'cpd00135', 'bigg.metabolite': 'hcys__L'}
{'metanetx.chemical': 'MNXM2449', 'seed.compound': 'cpd08636', 'bigg.metabolite': 'dhptd'}
{'metanetx.chemical': 'MNXM423', 'seed.compound': 'cpd00068', 'bigg.metabolite': 'itp'}
{'metanetx.chemical': 'MNXM495', 'seed.compound': 'cpd00090', 'bigg.metabolite': 'idp'}
{'metanetx.chem

{'metanetx.chemical': 'MNXM8669', 'seed.compound': 'cpd15605', 'bigg.metabolite': 'gly_phe__L'}
{'metanetx.chemical': 'MNXM8669', 'seed.compound': 'cpd15605', 'bigg.metabolite': 'glyphe'}
{'metanetx.chemical': 'MNXM97', 'seed.compound': 'cpd00066', 'bigg.metabolite': 'phe__L'}
{'metanetx.chemical': 'MNXM1177', 'seed.compound': 'cpd01977', 'bigg.metabolite': '4pasp'}
{'metanetx.chemical': 'MNXM713', 'seed.compound': 'cpd00395', 'bigg.metabolite': 'Lcyst'}
{'metanetx.chemical': 'MNXM114', 'seed.compound': 'cpd00129', 'bigg.metabolite': 'pro__L'}
{'metanetx.chemical': 'MNXM27', 'seed.compound': 'cpd00971', 'bigg.metabolite': 'na1'}
{'metanetx.chemical': 'MNXM114', 'seed.compound': 'cpd00129', 'bigg.metabolite': 'pro__L'}
{'metanetx.chemical': 'MNXM27', 'seed.compound': 'cpd00971', 'bigg.metabolite': 'na1'}
{'metanetx.chemical': 'MNXM4511', 'seed.compound': 'cpd15422', 'bigg.metabolite': 'cdpdtdec7eg'}
{'metanetx.chemical': 'MNXM366', 'seed.compound': 'cpd00493', 'bigg.metabolite': 'pyam5p

{'metanetx.chemical': 'MNXM420', 'seed.compound': 'cpd00184', 'bigg.metabolite': 'thymd'}
{'metanetx.chemical': 'MNXM420', 'seed.compound': 'cpd00184', 'bigg.metabolite': 'thymd'}
{'metanetx.chemical': 'MNXM92', 'seed.compound': 'cpd00078', 'bigg.metabolite': 'succoa'}
{'metanetx.chemical': 'MNXM1508', 'seed.compound': 'cpd03762', 'bigg.metabolite': 'bz12diol'}
{'metanetx.chemical': 'MNXM181', 'seed.compound': 'cpd00077', 'bigg.metabolite': 'catechol'}
{'metanetx.chemical': 'MNXM394', 'seed.compound': 'cpd00357', 'bigg.metabolite': 'dttp'}
{'metanetx.chemical': 'MNXM152', 'seed.compound': 'cpd00297', 'bigg.metabolite': 'dtdp'}
{'metanetx.chemical': 'MNXM701', 'seed.compound': 'cpd02691', 'bigg.metabolite': '3hmbcoa'}
{'metanetx.chemical': 'MNXM524', 'seed.compound': 'cpd02124', 'bigg.metabolite': '2maacoa'}
{'metanetx.chemical': 'MNXM826', 'seed.compound': 'cpd02501', 'bigg.metabolite': 'dhpppn'}
{'metanetx.chemical': 'MNXM337', 'seed.compound': 'cpd00216', 'bigg.metabolite': 'chor'}
{

{'metanetx.chemical': 'MNXM465', 'seed.compound': 'cpd00343', 'bigg.metabolite': 'cbasp'}
{'metanetx.chemical': 'MNXM874', 'seed.compound': 'cpd00939', 'bigg.metabolite': '4ahmmp'}
{'metanetx.chemical': 'MNXM9', 'seed.compound': 'cpd00009', 'bigg.metabolite': 'pi'}
{'metanetx.chemical': 'MNXM353', 'seed.compound': 'cpd00227', 'bigg.metabolite': 'hom__L'}
{'metanetx.chemical': 'MNXM361', 'seed.compound': 'cpd00346', 'bigg.metabolite': 'aspsa'}
{'metanetx.chemical': 'MNXM1805', 'seed.compound': 'cpd09027', 'bigg.metabolite': '5houdic'}
{'metanetx.chemical': 'MNXM1805', 'seed.compound': 'cpd09027', 'bigg.metabolite': 'houdic'}
{'metanetx.chemical': 'MNXM438', 'seed.compound': 'cpd00333', 'bigg.metabolite': 'cinnm'}
{'metanetx.chemical': 'MNXM1561', 'seed.compound': 'cpd02740', 'bigg.metabolite': 'gdptp'}
{'metanetx.chemical': 'MNXM214', 'seed.compound': 'cpd00650', 'bigg.metabolite': 'b2coa'}
{'metanetx.chemical': 'MNXM162233', 'seed.compound': 'cpd00175', 'bigg.metabolite': 'udpacgal'}
{

{'metanetx.chemical': 'MNXM172', 'seed.compound': 'cpd00199', 'bigg.metabolite': 'sucsal'}
{'metanetx.chemical': 'MNXM145761', 'seed.compound': 'cpd08375', 'bigg.metabolite': 'codscl8x'}
{'metanetx.chemical': 'MNXM145761', 'seed.compound': 'cpd08375', 'bigg.metabolite': 'copre8'}
{'metanetx.chemical': 'MNXM55375', 'seed.compound': 'cpd00155', 'bigg.metabolite': 'glycogen'}
{'metanetx.chemical': 'MNXM8669', 'seed.compound': 'cpd15605', 'bigg.metabolite': 'gly_phe__L'}
{'metanetx.chemical': 'MNXM8669', 'seed.compound': 'cpd15605', 'bigg.metabolite': 'glyphe'}
{'metanetx.chemical': 'MNXM1148', 'seed.compound': 'cpd00404', 'bigg.metabolite': 'orn__D'}
{'metanetx.chemical': 'MNXM8666', 'seed.compound': 'cpd15603', 'bigg.metabolite': 'gly_cys__L'}
{'metanetx.chemical': 'MNXM2846', 'seed.compound': 'cpd01197', 'bigg.metabolite': 'nebari'}
{'metanetx.chemical': 'MNXM2342', 'seed.compound': 'cpd00360', 'bigg.metabolite': 'purine'}
{'metanetx.chemical': 'MNXM91053', 'seed.compound': 'cpd15330', 

{'metanetx.chemical': 'MNXM291', 'seed.compound': 'cpd00180', 'bigg.metabolite': 'oxa'}
{'metanetx.chemical': 'MNXM1214', 'seed.compound': 'cpd00706', 'bigg.metabolite': 'mercppyr'}
{'metanetx.chemical': 'MNXM612', 'seed.compound': 'cpd01092', 'bigg.metabolite': 'alltn'}
{'metanetx.chemical': 'MNXM7127', 'seed.compound': 'cpd11482', 'bigg.metabolite': '3hdecACP'}
{'metanetx.chemical': 'MNXM26616', 'seed.compound': 'cpd11487', 'bigg.metabolite': '3odecACP'}
{'metanetx.chemical': 'MNXM1727', 'seed.compound': 'cpd01912', 'bigg.metabolite': 'fuc1p__L'}
{'metanetx.chemical': 'MNXM175', 'seed.compound': 'cpd00082', 'bigg.metabolite': 'fru'}
{'metanetx.chemical': 'MNXM1422', 'seed.compound': 'cpd02843', 'bigg.metabolite': 'eig3p'}
{'metanetx.chemical': 'MNXM365', 'seed.compound': 'cpd02851', 'bigg.metabolite': 'aicar'}
{'metanetx.chemical': 'MNXM1897', 'seed.compound': 'cpd04242', 'bigg.metabolite': 'd5kg'}
{'metanetx.chemical': 'MNXM271', 'seed.compound': 'cpd00238', 'bigg.metabolite': 's7p'

{'metanetx.chemical': 'MNXM39', 'seed.compound': 'cpd00047', 'bigg.metabolite': 'for'}
{'metanetx.chemical': 'MNXM434', 'seed.compound': 'cpd00460', 'bigg.metabolite': '2saa'}
{'metanetx.chemical': 'MNXM90703', 'seed.compound': 'cpd00166', 'bigg.metabolite': 'adocbl'}
{'metanetx.chemical': 'MNXM107505', 'seed.compound': 'cpd15378', 'bigg.metabolite': '4hba'}
{'metanetx.chemical': 'MNXM100', 'seed.compound': 'cpd00283', 'bigg.metabolite': 'grdp'}
{'metanetx.chemical': 'MNXM91788', 'seed.compound': 'cpd06711', 'bigg.metabolite': 'hmpscoa'}
{'metanetx.chemical': 'MNXM32179', 'seed.compound': 'cpd15538', 'bigg.metabolite': 'pg160'}
{'metanetx.chemical': 'MNXM58', 'seed.compound': 'cpd00048', 'bigg.metabolite': 'so4'}
{'metanetx.chemical': 'MNXM81221', 'seed.compound': 'cpd11830', 'bigg.metabolite': 'alpro'}
{'metanetx.chemical': 'MNXM114091', 'seed.compound': 'cpd02625', 'bigg.metabolite': '1p3h5c'}
{'metanetx.chemical': 'MNXM157', 'seed.compound': 'cpd00116', 'bigg.metabolite': 'meoh'}
{'

{'metanetx.chemical': 'MNXM29072', 'seed.compound': 'cpd11477', 'bigg.metabolite': 'tpalm2eACP'}
{'metanetx.chemical': 'MNXM242', 'seed.compound': 'cpd00105', 'bigg.metabolite': 'rib__D'}
{'metanetx.chemical': 'MNXM3932', 'seed.compound': 'cpd15414', 'bigg.metabolite': 'btnso'}
{'metanetx.chemical': 'MNXM213', 'seed.compound': 'cpd00226', 'bigg.metabolite': 'hxan'}
{'metanetx.chemical': 'MNXM8670', 'seed.compound': 'cpd11588', 'bigg.metabolite': 'gly_pro__L'}
{'metanetx.chemical': 'MNXM593', 'seed.compound': 'cpd00408', 'bigg.metabolite': 'pant__R'}
{'metanetx.chemical': 'MNXM959', 'seed.compound': 'cpd00712', 'bigg.metabolite': '2dhp'}
{'metanetx.chemical': 'MNXM813', 'seed.compound': 'cpd00331', 'bigg.metabolite': 'acon_C'}
{'metanetx.chemical': 'MNXM969', 'seed.compound': 'cpd03671', 'bigg.metabolite': 'gam1p'}
{'metanetx.chemical': 'MNXM1531', 'seed.compound': 'cpd01997', 'bigg.metabolite': 'dmbzid'}
{'metanetx.chemical': 'MNXM82173', 'seed.compound': 'cpd00557', 'bigg.metabolite':

{'metanetx.chemical': 'MNXM54163', 'seed.compound': 'cpd15469', 'bigg.metabolite': 'gagicolipa'}
{'metanetx.chemical': 'MNXM75132', 'seed.compound': 'cpd15549', 'bigg.metabolite': 'phhlipa'}
{'metanetx.chemical': 'MNXM56168', 'seed.compound': 'cpd15488', 'bigg.metabolite': 'hphhlipa'}
{'metanetx.chemical': 'MNXM75133', 'seed.compound': 'cpd15550', 'bigg.metabolite': 'phphhlipa'}
{'metanetx.chemical': 'MNXM128', 'seed.compound': 'cpd00063', 'bigg.metabolite': 'ca2'}
{'metanetx.chemical': 'MNXM58984', 'seed.compound': 'cpd15492', 'bigg.metabolite': 'kphphhlipa'}
{'metanetx.chemical': 'MNXM2255', 'seed.compound': 'cpd00030', 'bigg.metabolite': 'mn2'}



### Check for multiplicatives among the Bigg IDs

In [7]:
dupl_bigg = {}
for met in model.metabolites:
    if "bigg.metabolite" not in met.annotation.keys():
        continue
    if met.id in ["cpd00261_c0", "cpd06227_c0", "cpd03572_c0", "cpd02446_c0","cpd02572_c0", "cpd01466_c0" ]:
        pass
    else:    
        if met.annotation["bigg.metabolite"] not in dupl_bigg.keys():
            dupl_bigg.setdefault(met.annotation["bigg.metabolite"],[met.id])
        else:
            dupl_bigg[met.annotation["bigg.metabolite"]].append(met.id)

['MNXM421_c0']

['MNXM1160_c0']

['MNXM11_c0']

['MNXM1_c0']

['MNXM965_c0']

...

['MNXM54163_c0']

['MNXM75132_c0']

['MNXM56168_c0']

['MNXM75133_c0']

['MNXM58984_c0']

In [8]:
for ls in dupl_bigg.values():
    if len(ls) > 1:
        print(ls)

['MNXM11_c0', 'MNXM11_e0']
['MNXM1_c0', 'MNXM1_e0']
['MNXM9_c0', 'MNXM9_e0']
['MNXM213_c0', 'MNXM213_e0']
['MNXM2_c0', 'MNXM2_e0']
['MNXM42_c0', 'MNXM42_e0']
['MNXM61_c0', 'MNXM61_e0']
['MNXM212_e0', 'MNXM212_c0']
['MNXM53_c0', 'MNXM53_e0']
['MNXM93_e0', 'MNXM93_c0']
['MNXM89557_c0', 'MNXM89557_e0']
['MNXM13_c0', 'MNXM13_e0']
['MNXM107_e0', 'MNXM107_c0']
['MNXM89689_e0', 'MNXM89689_c0']
['MNXM165_c0', 'MNXM165_e0']
['MNXM41_c0', 'MNXM41_e0']
['MNXM131_c0', 'MNXM131_e0']
['MNXM15_c0', 'MNXM15_e0']
['MNXM4_c0', 'MNXM4_e0']
['MNXM78_c0', 'MNXM78_e0']
['MNXM1191_c0', 'MNXM1191_e0']
['MNXM334_c0', 'MNXM334_e0']
['MNXM289_c0', 'MNXM289_e0']
['MNXM55269_c0', 'MNXM55269_e0']
['MNXM58_c0', 'MNXM58_e0']
['MNXM338_c0', 'MNXM338_e0']
['MNXM704_c0', 'MNXM704_e0']
['MNXM25_e0', 'MNXM25_c0']
['MNXM8825_e0', 'MNXM8825_c0']
['MNXM89612_c0', 'MNXM89612_e0']
['MNXM118_c0', 'MNXM118_e0']
['MNXM124_c0', 'MNXM124_e0']
['MNXM282_c0', 'MNXM282_e0']
['MNXM66_c0', 'MNXM66_e0']
['MNXM39_c0', 'MNXM39_e0']
['MNXM1

In [9]:
for ls in dupl_bigg.values():
    if len(ls) > 1:
        #print(ls)
        x=[]
        y=[]
        for members in ls: 
            main_id, comp =members.split("_")
            x.append(main_id)
            y.append(comp)
        if not x[0]==x[1]:
            print(x,y)
            for pair in zip(x,y):
                print(len(model.metabolites.get_by_id(pair[0]+"_"+pair[1]).reactions))

### Change the MNXM IDs to Bigg IDs

In [10]:
for met in model.metabolites:
    
    if "bigg.metabolite" not in met.annotation.keys():
        continue
    if met.id in ["cpd00261_c0", "cpd06227_c0", "cpd03572_c0", "cpd02446_c0","cpd02572_c0", "cpd01466_c0" ]:
        pass
    else:    
    #if met.id.startswith('cpd'):
        split, compartment=met.id.split("_")
        met.id
    
        met.id=met.annotation['bigg.metabolite']+ '_' + compartment
        model.repair()
        print(met.id)

'MNXM421_c0'

4abz_c0


'MNXM1160_c0'

6hmhptpp_c0


'MNXM11_c0'

...

ca2_e0


'MNXM58984_c0'

kphphhlipa_c0


'MNXM2255_e0'

mn2_e0


### Save draft with mapped Bigg IDs

In [11]:
#write_sbml_model(model, "/Users/lizrad/Dev/iVnat/iVnat.xml")